<a href="https://colab.research.google.com/github/ajlewisl23/SENSE_2023_GEE/blob/master/Frozen_Shorelines_ML_SENTINEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install geehydro

  Preparing metadata (setup.py) ... done
  Created wheel for geehydro: filename=geehydro-0.2.0-py2.py3-none-any.whl size=10125 sha256=ac34c63497109e6ed0dfc951408336d50b1f538ca3e9ce8710e7aea53295b3ac
  Stored in directory: /root/.cache/pip/wheels/90/ad/c9/ab38b841cd7f4dc8070c1ceb90810f6b7228daa3a4081f4880
Successfully built geehydro


In [ ]:
# Import earthengine API
import ee
# Authenticate and initialise
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=Nr9h3tPDI42nZJhJMKT7U2p7p8PlE2cZGfs2dEAaoUc&tc=HE3KnZ_YUK4ljed0D32OUSMMh0LRkAlIvN0OM7p6e6I&cc=eaSqbhihUZ0XpsZMjpILsHrhPum8yYFY1kR8MmdgHlw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlA-BeLigUvY785J5yjZNcVDiFNQoeX-DzgbZKmgcGFJyi8tRQMaBk

Successfully saved authorization token.


In [ ]:
import folium
import geehydro

In [ ]:
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
#print(l7.size().getInfo())
BL_coord = [102.42688047442051,70.52945533066296]
TR_coord = [133.05676328692053,76.21969845819945]
spec_region = ee.Geometry.Rectangle(102.42688047442051,70.52945533066296,133.05676328692053,76.21969845819945)

In [ ]:
l7_filtered = l7.filterBounds(spec_region).filterDate('1999-01-01', '2023-12-31').filterMetadata('CLOUD_COVER','less_than',10)
l8_filtered = l8.filterBounds(spec_region).filterDate('1999-01-01', '2023-12-31').filterMetadata('CLOUD_COVER','less_than',10)
l9_filtered = l9.filterBounds(spec_region).filterDate('1999-01-01', '2023-12-31').filterMetadata('CLOUD_COVER','less_than',10)
#print(l7_filtered.size().getInfo())

In [ ]:
merged_7_9 = l7_filtered.merge(l8_filtered).merge(l9_filtered)
merged_7_9.size().getInfo()

4360

In [ ]:
merged_7_9.filterDate('2023-06-01', '2023-09-30').size().getInfo()
print(l7.first().bandNames())

ee.List({
  "functionInvocationValue": {
    "functionName": "Image.bandNames",
    "arguments": {
      "image": {
        "functionInvocationValue": {
          "functionName": "Collection.first",
          "arguments": {
            "collection": {
              "functionInvocationValue": {
                "functionName": "ImageCollection.load",
                "arguments": {
                  "id": {
                    "constantValue": "LANDSAT/LE07/C02/T1_L2"
                  }
                }
              }
            }
          }
        }
      }
    }
  }
})


In [145]:
merged_7_9_filtered = merged_7_9.filterBounds(spec_region).filterDate('2016-06-01', '2016-08-31').filterMetadata('CLOUD_COVER','less_than',50)
bands_7_9 = merged_7_9_filtered.select('SR_B1','SR_B2','SR_B3','SR_B4','SR_B5')

In [146]:
map = folium.Map(location=[75, 117], zoom_start=6)
image_2015 = bands_7_9.median()
map.addLayer(image_2015, {'bands': ['SR_B3','SR_B2','SR_B1']}, 'image_2015')
map

In [131]:
merged_7_9_filtered = merged_7_9.filterBounds(spec_region).filterDate('2023-07-01', '2023-08-31').filterMetadata('CLOUD_COVER','less_than',10)
bands_7_9 = merged_7_9_filtered.select('SR_B1','SR_B2','SR_B3','SR_B4','SR_B5')

In [132]:
map = folium.Map(location=[75, 117], zoom_start=6)
image_2023 = bands_7_9.median()
map.addLayer(image_2023, {'bands': ['SR_B3','SR_B2','SR_B1']}, {min:0, max:255}, 'image_2023')
map

In [112]:
region = ee.Geometry.Rectangle(110, 73, 128, 74.5)
map = folium.Map(location=[73.75, 119],zoom_start=5)

map.addLayer(ee.Image().paint(region, 0, 2), {}, 'region')
map

In [113]:
# Make the training dataset.
training = image.sample(region = region,scale= 30,  numPixels= 5000)

# Instantiate the clusterer and train it.
clusterer = ee.Clusterer.wekaKMeans(4).train(training)

# Cluster the input using the trained clusterer.
result = image.cluster(clusterer)

# Display the clusters with random colors.

map.addLayer(result.randomVisualizer(), {}, 'clusters')
folium.LayerControl().add_to(map)
map

In [119]:
NDWI_2023 = image_2023.normalizedDifference(['SR_B2', 'SR_B4']).rename(['NDWI_2023'])
NDWI_2015 = image_2015.normalizedDifference(['SR_B2', 'SR_B4']).rename(['NDWI_2015'])
#NWDI_1 = (image.select('SR_B2').subtract(image.select('SR_B4')).divide(image.select('SR_B2').add(image.select('SR_B4'))).rename(['NWDI_1']))
#map.addLayer(NWDI_1, {'bands': ['NWDI_1']}, {min:0, max:255})
#map.addLayer(image, {'bands': ['SR_B2']})
#map.addLayer(NDWI)
#map

In [122]:
#map = folium.Map(location=[75, 117], zoom_start=6)
ndwiThreshold = NDWI_2023.gte(0.0)
ndwiMask = ndwiThreshold.updateMask(ndwiThreshold)
map.addLayer(ndwiThreshold)
map.addLayer(ndwiMask)
map